# predicting Rossman sale


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder,LabelEncoder


In [2]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [3]:
train = pd.read_csv("data/train.csv")
store = pd.read_csv("data/store.csv") 

C:\Users\mhnaj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data exploration

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637774 entries, 0 to 637773
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           637774 non-null  object 
 1   Store          618473 non-null  float64
 2   DayOfWeek      618757 non-null  float64
 3   Sales          618747 non-null  float64
 4   Customers      618683 non-null  float64
 5   Open           618588 non-null  float64
 6   Promo          618580 non-null  float64
 7   StateHoliday   618520 non-null  object 
 8   SchoolHoliday  618437 non-null  float64
dtypes: float64(7), object(2)
memory usage: 43.8+ MB


In [5]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


### Missing value in Store.csv

In [6]:
train['Date'] = pd.to_datetime(train['Date'])

In [7]:
def fillna_mean(df,columns):
    for col in columns:
        mean_value = int(df[col].mean())
        df.loc[:,col].fillna(value=mean_value,inplace=True)
    return df

def fillna_most(df,columns):
    for col in columns:
        most_value = df[col].value_counts().idxmax()
        df.loc[:,col].fillna(value=most_value,inplace=True)
    return df
columns_mean = ['CompetitionOpenSinceMonth',
                'CompetitionOpenSinceYear',
                'CompetitionDistance',
                'Promo2SinceWeek',
                'Promo2SinceYear'
                 ] 

store = fillna_mean(store,columns_mean)

columns_most = ['PromoInterval']
stor = fillna_most(store,columns_most)


store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1115 non-null   float64
 4   CompetitionOpenSinceMonth  1115 non-null   float64
 5   CompetitionOpenSinceYear   1115 non-null   float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            1115 non-null   float64
 8   Promo2SinceYear            1115 non-null   float64
 9   PromoInterval              1115 non-null   object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


Drop rows that has no sales (zero or null value)

In [8]:
train.dropna(subset=['Sales'],inplace=True)
#train.drop(columns=['Customers'],inplace=True)
train.dropna(subset=['Store'],inplace=True) 

train= train[train['Sales']>0]

### Missing value in train.csv


In [9]:
columns_mean = ['DayOfWeek','Customers']
train = fillna_mean(train,columns_mean)

columns_most = ['Promo','SchoolHoliday','StateHoliday']
train = fillna_most(train,columns_most)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497376 entries, 27 to 637773
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           497376 non-null  datetime64[ns]
 1   Store          497376 non-null  float64       
 2   DayOfWeek      497376 non-null  float64       
 3   Sales          497376 non-null  float64       
 4   Customers      497376 non-null  float64       
 5   Open           482366 non-null  float64       
 6   Promo          497376 non-null  float64       
 7   StateHoliday   497376 non-null  object        
 8   SchoolHoliday  497376 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 37.9+ MB


Merge 'train' and 'store' together

In [10]:
train_full = pd.merge(train,store, on='Store', how='inner')

In [11]:
train_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497376 entries, 0 to 497375
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Date                       497376 non-null  datetime64[ns]
 1   Store                      497376 non-null  float64       
 2   DayOfWeek                  497376 non-null  float64       
 3   Sales                      497376 non-null  float64       
 4   Customers                  497376 non-null  float64       
 5   Open                       482366 non-null  float64       
 6   Promo                      497376 non-null  float64       
 7   StateHoliday               497376 non-null  object        
 8   SchoolHoliday              497376 non-null  float64       
 9   StoreType                  497376 non-null  object        
 10  Assortment                 497376 non-null  object        
 11  CompetitionDistance        497376 non-null  float64 

### Data Cleaning


In [12]:
train_full['Open'].value_counts()

1.0    482366
Name: Open, dtype: int64

In [13]:
#train_full.drop(columns=['Open'],inplace=True)

In [14]:
train_full['StateHoliday'].value_counts()

0      445761
0.0     51088
a         401
b          91
c          35
Name: StateHoliday, dtype: int64

In [15]:
train_full.loc[train_full['StateHoliday']==0,'StateHoliday']='0'
train_full.loc[train_full['StateHoliday']=='0','StateHoliday']='0'

train_full['StateHoliday'] = LabelEncoder().fit_transform(train_full['StateHoliday'])

### Define new feutures

In [16]:
train_full['CompetitionOpen'] = 12*(train_full.loc[:,'Date'].dt.year.max()-train_full.loc[:,'CompetitionOpenSinceYear'])-train_full.loc[:,'CompetitionOpenSinceMonth']    
train_full['Promo2Open'] = 52*(train_full.loc[:,'Date'].dt.year.max()-train_full.loc[:,'Promo2SinceYear'])-train_full.loc[:,'Promo2SinceWeek']

#### Customer per Store per Day

In [52]:
cutomer_store = train_full.groupby('Store').agg(cust_st=('Customers','mean'))
open_store = train_full.groupby('Store').agg(open_st=('Open','count'))
#customer_day_store = (cutomer_store / open_store)
#cutomer_store
#train_full = pd.merge(train_full, customer_day_store, how='left', on=['Store'])
#train_full['CustomersPerDay']

In [59]:
(cutomer_store / open_store)

,cust_st,open_st
Store,,
1.0,NaN,NaN
2.0,NaN,NaN
3.0,NaN,NaN
4.0,NaN,NaN
5.0,NaN,NaN
...,...,...
1111.0,NaN,NaN
1112.0,NaN,NaN
1113.0,NaN,NaN


In [49]:
customer_day_store = (cutomer_store / open_store)
train_full = pd.merge(train_full, customer_day_store.reset_index(name='CustomersPerDay'), how='left', on=['Store'])

TypeError: reset_index() got an unexpected keyword argument 'name'

In [18]:
#customer_day_store = (cutomer_store / open_store)

#train_full = pd.merge(train_full, customer_day_store, how='left', on=['Store'])


#### Promo yesterday and tomorrow

In [45]:
#train_full['PromoTomorrow'] = train_full['Promo'].shift(-1)
#train_full['PromoYesterday'] = train_full['Promo'].shift(1)


1

#### Holiday last week , this week and next week

In [19]:
import datetime
def holidays_week(df,start,end,start_past=False):
    holidays = []
    holidays_index = []
    for index,value in df.groupby('Date').sum().iterrows():
        first = index + datetime.timedelta(days=start)
        if start_past:
            first = index - datetime.timedelta(days=start)
        last =  index + datetime.timedelta(days=end)
        school_holidays = sum((df.groupby('Date').sum()[first:last])['SchoolHoliday'])
        state_holidays = sum((df.groupby('Date').sum()[first:last])['StateHoliday'])
        holidays.append(school_holidays+state_holidays)
        holidays_index.append(index)
    return holidays,holidays_index

holidays_last_week,holidays_last_index=holidays_week(train_full,7,1,start_past=True)
holidays_this_week,holidays_this_index=holidays_week(train_full,0,7)
holidays_next_week,holidays_next_index=holidays_week(train_full,7,15)

In [20]:
temp_df = pd.DataFrame({'HolidaysLastWeek':holidays_last_week, 'Date': holidays_last_index})
train_full = pd.merge(train_full, temp_df, on=['Date'])

temp_df = pd.DataFrame({'HolidaysThisWeek':holidays_this_week, 'Date': holidays_this_index})
train_full = pd.merge(train_full, temp_df, on=['Date'])

temp_df = pd.DataFrame({'HolidaysNextWeek':holidays_next_week, 'Date': holidays_next_index})
train_full = pd.merge(train_full, temp_df, on=['Date'])



### Dealing with categorical data

In [21]:
def get_dummies(df,columns):
    for col in columns:
        if col in df.columns:
            df = pd.get_dummies(df, columns = [col])
    return df

train_full = get_dummies(train_full,['StoreType','Assortment','PromoInterval'])

freq = train_full.groupby('Store').size()/len(train_full)
train_full.loc[:,'Store_freq'] = train_full.loc[:,'Store'].map(freq)

In [22]:
train_full.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,...,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",Store_freq
0,2013-01-01,353.0,2.0,3139.0,820.0,1.0,0.0,1,1.0,900.0,...,1,0,0,0,1,0,1,0,0,0.001074
1,2013-01-01,335.0,2.0,2401.0,482.0,1.0,0.0,1,1.0,90.0,...,1,0,0,1,0,0,0,1,0,0.001100
2,2013-01-01,512.0,2.0,2646.0,625.0,1.0,0.0,1,1.0,590.0,...,1,0,0,0,1,0,0,0,1,0.000995
3,2013-01-01,494.0,2.0,3113.0,527.0,1.0,0.0,1,1.0,1260.0,...,1,0,0,1,0,0,0,1,0,0.001094
4,2013-01-01,530.0,2.0,2907.0,532.0,1.0,0.0,1,1.0,18160.0,...,0,0,0,0,0,1,0,1,0,0.001052


### Split Data: Train and Test

In [38]:
X = train_full.drop(columns=['Sales','Date','Open','Customers','Store','CompetitionOpenSinceYear','CompetitionOpenSinceMonth','Promo2SinceYear','Promo2SinceWeek'])
y = train_full['Sales']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    X, y, test_size=0.33)

### Models:

### Random Forest

In [40]:
from sklearn.ensemble import RandomForestRegressor 
rf = RandomForestRegressor(max_depth=30)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [35]:
rf_rmspe = metric(predictions,y_test.to_numpy())

In [36]:
rf_rmspe

18.25040278702535

In [ ]:
'''
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [30, 50, 80],
    'max_features': [3, 5,7],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

grid_search.best_estimator_
best_grid = grid_search.best_estimator_
predictions = best_grid.predict(X_test)

rf_rmspe = metric(predictions,y_test.to_numpy())
'''